## Documentation

To read more about the search API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Inserting documents

In [ ]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

Let's index the documents sequentially.

In [ ]:
import json
from tqdm import tqdm


dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='my_index', body=document)

## Searching

### 1. Leaf clauses

#### 1.1. term query

Let's use the `Query DSL` language to construct a query that will find any document that was created on `2024-09-22`

In [ ]:
response = es.search(
    index='my_index',
    body={
        "query": {
            "term": {
                "created_on": "2024-09-22"
            }
        }
    }
)

n_hits = response['hits']['total']['value']
print(f"Found {n_hits} documents in my_index")

To retrieve the document just use the `hits` dictionary like this.

In [ ]:
retrieved_documents = response['hits']['hits']
retrieved_documents

#### 1.2. match query

Now, let's search for any document that contains the word `document` in the text field.

In [ ]:
response = es.search(
    index='my_index',
    body={
        "query": {
            "match": {
                "text": "document"
            }
        }
    }
)

n_hits = response['hits']['total']['value']
print(f"Found {n_hits} documents in my_index")

In [ ]:
retrieved_documents = response['hits']['hits']
retrieved_documents

#### 1.3. range query

Let's find documents that were created before `2024-09-24`

In [ ]:
response = es.search(
    index='my_index',
    body={
        "query": {
            "range": {
                "created_on": {
                    "lte": "2024-09-23"
                }
            }
        }
    }
)

n_hits = response['hits']['total']['value']
print(f"Found {n_hits} documents in my_index")

In [ ]:
retrieved_documents = response['hits']['hits']
retrieved_documents

This is how you use the leaf clauses. Now, if you want to combine leaf clauses together, you do that with the compound clauses.

### 2. Compound clauses

Let's search for documents that meet the following criteria:
- Created on `2024-09-24`
- Have the word `third` in the text field.

In [ ]:
response = es.search(
    index='my_index',
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": "third"
                        }
                    },
                    {
                        "range": {
                            "created_on": {
                                "gte": "2024-09-24",
                                "lte": "2024-09-24"
                            }
                        }
                    }
                ]
            }
        }
    }
)

n_hits = response['hits']['total']['value']
print(f"Found {n_hits} documents in my_index")

In [ ]:
retrieved_documents = response['hits']['hits']
retrieved_documents

With the compound clause, we were to combine two leaf clauses to find a specific document.